In [ ]:
import fiona
from fiona.transform import transform_geom
import os
import logging

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(message)s')

# Target coordinate reference system
TARGET_CRS = 'EPSG:3857'

# Function to reproject a shapefile
def reproject_shapefile(input_shp):
    output_shp = f"reprojected_{os.path.basename(input_shp)}"

    with fiona.open(input_shp, 'r') as source:
        source_crs = source.crs
        schema = source.schema

        with fiona.open(
            output_shp, 'w',
            driver=source.driver,
            crs=TARGET_CRS,
            schema=schema
        ) as target:
            for feature in source:
                transformed_geom = transform_geom(source_crs, TARGET_CRS, feature['geometry'])
                feature['geometry'] = transformed_geom
                target.write(feature)

    logging.info(f"Reprojected: {input_shp} -> {output_shp}")

# Find and process shapefiles in the current directory
for file in os.listdir():
    if file.endswith('.shp'):
        reproject_shapefile(file)

logging.info("Processing complete.")


In [3]:
import fiona
from fiona.transform import transform_geom
import os
import logging
import json

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(message)s')

# Target coordinate reference system
TARGET_CRS = 'EPSG:3857'

# Function to reproject a shapefile and output ArcGIS JSON
def reproject_shapefile(input_shp):
    output_json = f"reprojected_{os.path.splitext(os.path.basename(input_shp))[0]}.json"
    features = []

    with fiona.open(input_shp, 'r') as source:
        source_crs = source.crs

        for feature in source:
            transformed_geom = transform_geom(source_crs, TARGET_CRS, feature['geometry'])
            arcgis_feature = {
                "geometry": {
                    "rings": transformed_geom['coordinates'] if transformed_geom['type'] == 'Polygon' else [],
                    "spatialReference": {"wkid": 3857}
                },
                "attributes": dict(feature['properties'])  # Ensure properties are serializable
            }
            features.append(arcgis_feature)

    with open(output_json, 'w') as json_file:
        json.dump(features, json_file, indent=4)

    logging.info(f"Reprojected and saved as ArcGIS JSON: {output_json}")

# Find and process shapefiles in the current directory
for file in os.listdir():
    if file.endswith('.shp'):
        reproject_shapefile(file)

logging.info("Processing complete.")



Reprojected and saved as ArcGIS JSON: reprojected_PointsOut.json
Processing complete.
